### Analysis Brief: 
* In Q3, both Guest Retention and Acquisition started trending down. The CRM Guest Analytics team wanted to better understand the drivers of this decline.
 
### Executive Summary
* **Ecom & non-core products shine in Q2**. Q2 driven by exceptional ecom performance across non-core products early in quarter
* **Keep an eye out for Q3 Guest health metrics**. 2021 Q2 to Q3, across both acquisition & retention, has been trending down… with the effects amplified to start Q3 by the flat-to-positive performance observed in 2019
* **Are we becoming more reliant on other products?**. Core showing signs of rebounding as of week 33, and has a performance trend that more closely mirror’s 2019. However, core volume is not large enough to move the needle above & beyond the drop seen across non-core.
* **A soft start to Q3**. Business performance showing softness in Q3 due to the bottoming out of ecom, which has fallen from weekly +130-150% 2yy early in Q2 (across both acquisition & retention) to just 0-30% yy in weeks 30-33. That drop has exposed store, which has fallen from -5% yy to start Q2 to -50% 2yy weeks 30-33 (across both acquisition & retention)
* **Are customers buying less colours & sizes?**.

### Analysis

## Part 1: Get data
* load libraries
* connect to snowflake
* define sql query
* run and store query

In [2]:
# Load required libraries

import pandas as  pd
from datetime import date 
import pyodbc

In [3]:
# Create snowflake connection function
def connect_snowflake(
        user, 
        server='lululemon_martech_replication.east-us-2.azure.snowflakecomputing.com',
        role='SNOWFLAKE-LLL.MARTECH.GUEST_ANALYTICS_DB.DEVELOPER',
        warehouse='GUEST_ANALYTICS_S_DW',
        database='GUEST_ANALYTICS_DB',
        schema = 'TEMP_SCH'):

    """

    What: 
        Establish connection with a Snowflake instance

    Args:
        user (str): lululemon's user name
        server (str, optional): Server URL. Defaults to 'lululemon_martech_replication.east-us-2.azure.snowflakecomputing.com'.
        role (str, optional): Role used to connect to Snowflake. Defaults to 'SNOWFLAKE-LLL.MARTECH.GUEST_ANALYTICS_DB.DEVELOPER'.
        warehouse (str, optional): Warehouse name. Defaults to 'GUEST_ANALYTICS_S_DW'.
        database (str, optional): Database name. Defaults to 'GUEST_ANALYTICS_DB'.
        schema (str, optional): Schema name. Defaults to 'TEMP_SCH'.

    Returns:
        pyodbc.Connection: SQL connections

    """

    if user is None:
        print('Error: Snowflake user must be defined.')
        return False

    driver = '{SnowflakeDSIIDriver}'
    authenticator = 'externalbrowser'

    conn = pyodbc.connect (
        'DRIVER='+driver+
        ';SERVER='+server+
        ';ROLE='+role+
        ';WAREHOUSE='+warehouse+
        ';DATABASE='+database+
        ';SCHEMA='+schema+
        ';UID='+user+
        ';AUTHENTICATOR='+authenticator

    )

    return conn

# Store connection to snowflake as a variable
conn = connect_snowflake("dfayad@lululemon.com")

# Run it (this should prompt a snowflake web URL to open, testing & verifying the authentication)
conn

In [4]:
# Define sql query, run it using pre-defined creds
grouped_sql =("""WITH 

-- #CTE1: WOMENS TX BY CORE, NON CORE (CUSTOM DEFINITION), WITH PRODUCT INFO
  TX_FW AS (
    SELECT
      td.MASTER_GUEST_ID
      , td.TRANSACTION_ID
      , td.TXN_TIMESTAMP_PACIFIC
      , td.TXN_DATE_ID_PACIFIC
      , dd.FISCAL_WEEK
      , dd.FISCAL_YEAR_NUM
      , td.MASTER_ORDER_ORIGIN
      , td.SKU
      , td.CURRENCY_CODE
      , lower(pm.MMS_STYLE_DESCRIPTION) as MMS_STYLE_DESCRIPTION
      , lower(pm.MMS_DEPT_NAME3) as MMS_DEPT_NAME3
      , SPLIT_PART(lower(pm.FSO_COLORWAY_NAME), ' - ', 2) AS MMS_COLORWAY -- only get the part after the hyphen
      , lower(pm.FSO_COLORWAY_NAME) AS FSO_COLORWAY -- ORIGINAL
      , cs.SKU_COLOR_GROUP
      , cs.SKU_COLOR_CODE_DESC
      , cs.NUM_COLORS
      , cs.COLOR1
      , cs.COLOR2
      , cs.COLOR3
      , cs.COLOR4
      , cs.COLOR5
      , cs.SKU_SIZE AS FUS_SIZE
      , pm.MMS_SIZDSC as MMS_SIZE
      , CASE WHEN td.MASTER_GUEST_ID is null THEN 1 ELSE 0 END as no_id_guest
      , CASE WHEN contains(lower(pm.FSO_COLORWAY_NAME), 'blk') AND contains(lower(pm.MMS_STYLE_DESCRIPTION),'align') AND td.MMS_DEPT_NAME = 'LULU WOMENS' THEN 1 ELSE 0 END as MMS_BA_FLAG -- black align flag
      , CASE WHEN cs.SKU_COLOR_GROUP LIKE '%black%' AND contains(lower(pm.MMS_STYLE_DESCRIPTION),'align') AND td.MMS_DEPT_NAME = 'LULU WOMENS' THEN 1 ELSE 0 END as FUS_BA_FLAG -- black align flag 2
      , CASE WHEN contains(lower(pm.FSO_COLORWAY_NAME), 'blk') AND td.MMS_DEPT_NAME = 'LULU WOMENS' THEN 1 ELSE 0 END as MMS_B_FLAG -- black flag 1
      , CASE WHEN cs.SKU_COLOR_GROUP LIKE '%black%' AND pm.MMS_DEPT_NAME = 'LULU WOMENS' THEN 1 ELSE 0 END as FUS_B_FLAG -- black flag 2
      , CASE WHEN lower(pm.MMS_STYLE_DESCRIPTION) like '%align%pant%28%'
               or lower(pm.MMS_STYLE_DESCRIPTION) like '%align%pant%25%'
               or lower(pm.MMS_STYLE_DESCRIPTION) like '%wunder%train%25%'
               or lower(pm.MMS_STYLE_DESCRIPTION) like '%fast%free%25%'
             THEN 1 
             ELSE 0 
           END as WCORE_FLAG -- KEVINS DEFINITION
      , CASE WHEN td.TXN_SEQUENCE_ASC = 1 AND td.MMS_DEPT_NAME = 'LULU WOMENS' THEN 'ACQ' ELSE 'RET' END as TYPE
      , CASE WHEN td.MMS_DEPT_NAME = 'LULU WOMENS' THEN 1 ELSE 0 END as WOMENS

    FROM 
      MARTECH_DB.NA_ANALYTICS_SCH.FACT_TXN_DETAIL td
      LEFT JOIN MARTECH_DB.NA_ANALYTICS_SCH.DIM_PRODUCT_MASTER_MMS pm on td.SKU = pm.SKU
      LEFT JOIN GUEST_ANALYTICS_DB.EXPIRES_ON_20210131_SCH.FUSION_COLOURS_SIZES_QA cs on td.SKU = cs.SKU_ID_NUM
      LEFT JOIN MARTECH_DB.NA_LANDING_SCH.DIM_DATE dd on td.TXN_DATE_ID_PACIFIC = dd.DATE_ID

    WHERE 1 = 1
      AND dd.FISCAL_YEAR_NUM between 2015 and 2021
      AND dd.FISCAL_WEEK between 14 and 33
      AND td.TXN_SEQUENCE_ASC > 0 -- only look at txn_seq greater than 1, if removed will need to change TYPE logic
      AND td.MMS_DEPT_NAME = 'LULU WOMENS' -- WOMENS ONLY
    )

-- #CTE2: Add a feature - if customer ever bought a womens's core (wcore) product in time period, then 1 else 0
  ,TX_FW_WCORE_MAX AS (
    SELECT 
      t1.* -- just get all columns
      , max(t1.WCORE_FLAG) OVER (PARTITION BY t1.MASTER_GUEST_ID,t1.TYPE) as WCORE_COUNT
    FROM 
      TX_FW t1
    
    )

-- checking the nulls in mms and fusion and prod-tx level
/**
SELECT  x.FISCAL_YEAR_NUM
        , SUM(CASE WHEN x.SKU_COLOR_CODE_DESC IS NULL THEN 1 ELSE 0 END) AS fusion_nulls
        , SUM(CASE WHEN x.MMS_COLORWAY IS NULL THEN 1 ELSE 0 END) AS mms_colorway_nulls
        , SUM(CASE WHEN x.MMS_DEPT_NAME3 IS NULL THEN 1 ELSE 0 END) AS mms_dept_nulls
        , SUM(CASE WHEN x.SKU IS NOT NULL THEN 1 ELSE 0 END) as all_product_rows
        , ( SUM(CASE WHEN x.SKU_COLOR_CODE_DESC IS NULL THEN 1 ELSE 0 END) / COUNT(*) ) AS pcnt_fusion_nulls
        , ( SUM(CASE WHEN x.MMS_COLORWAY IS NULL THEN 1 ELSE 0 END) / SUM(CASE WHEN x.SKU IS NOT NULL THEN 1 ELSE 0 END) ) AS pcnt_mms_colorway_nulls
        , ( SUM(CASE WHEN x.MMS_DEPT_NAME3 IS NULL THEN 1 ELSE 0 END) / SUM(CASE WHEN x.SKU IS NOT NULL THEN 1 ELSE 0 END) ) AS pcnt_mms_dept_nulls

FROM    TX_FW x
GROUP BY x.FISCAL_YEAR_NUM
ORDER BY x.FISCAL_YEAR_NUM DESC; **/

-- LAST CALL
SELECT
  t2.FISCAL_YEAR_NUM
  , t2.FISCAL_WEEK
  , t2.MASTER_ORDER_ORIGIN
  , t2.MMS_DEPT_NAME3
  , t2.CURRENCY_CODE
  , t2.TYPE
  , t2.NUM_COLORS -- fusion
  , t2.MMS_COLORWAY -- mms definition
  , t2.SKU_COLOR_GROUP -- fusion definition
  , t2.MMS_SIZE -- mms definition
  , t2.FUS_SIZE -- fusion definition
  , t2.COLOR1 -- fusion
  , t2.COLOR2 -- fusion
  , t2.COLOR3 -- fusion
  , t2.COLOR4 -- fusion
  , t2.COLOR5 -- fusion
  , sum(t2.NO_ID_GUEST) as NO_ID_GUEST
  , count(distinct t2.MASTER_GUEST_ID) as GUEST_COUNT
  , count(distinct t2.TRANSACTION_ID) as TXN_COUNT
  , sum(t2.WCORE_FLAG) as TXN_WCORE
  , sum(CASE WHEN t2.WCORE_FLAG = 0 THEN 1 ELSE 0 END) as TXN_WREST
  , sum(t2.WCORE_COUNT) as GUEST_WCORE

FROM 
  TX_FW_WCORE_MAX t2
  
GROUP BY 
  t2.FISCAL_YEAR_NUM
  , t2.FISCAL_WEEK
  , t2.MASTER_ORDER_ORIGIN
  , t2.MMS_DEPT_NAME3
  , t2.CURRENCY_CODE
  , t2.TYPE
  , t2.NUM_COLORS -- fusion
  , t2.MMS_COLORWAY -- mms definition
  , t2.SKU_COLOR_GROUP -- fusion definition
  , t2.MMS_SIZE -- mms definition
  , t2.FUS_SIZE -- fusion definition
  , t2.COLOR1 -- fusion
  , t2.COLOR2 -- fusion
  , t2.COLOR3 -- fusion
  , t2.COLOR4 -- fusion
  , t2.COLOR5 -- fusion
  ;""")

# Run the sql query, store it as df
df = pd.read_sql(grouped_sql,conn)

# See a few rows of the dataset
df.head(5)

,FISCAL_YEAR_NUM,FISCAL_WEEK,MASTER_ORDER_ORIGIN,MMS_DEPT_NAME3,CURRENCY_CODE,TYPE,NUM_COLORS,MMS_COLORWAY,SKU_COLOR_GROUP,MMS_SIZE,...,COLOR2,COLOR3,COLOR4,COLOR5,NO_ID_GUEST,GUEST_COUNT,TXN_COUNT,TXN_WCORE,TXN_WREST,GUEST_WCORE
0,2018.0,18.0,STORE,womens shorts,CAD,RET,NaN,lcdl,None,4,...,None,None,None,None,0.0,6.0,6.0,0.0,6.0,4.0
1,2018.0,25.0,STORE,womens ss tops,USD,RET,NaN,blk/blk,None,6,...,None,None,None,None,0.0,156.0,158.0,0.0,162.0,85.0
2,2018.0,19.0,ECOM,womens sportsbras,CAD,RET,NaN,ikbl/blk,None,6,...,None,None,None,None,0.0,19.0,23.0,0.0,23.0,18.0
3,2018.0,28.0,STORE,womens ls tops,USD,RET,NaN,wht,None,6,...,None,None,None,None,0.0,229.0,237.0,0.0,242.0,120.0
4,2018.0,21.0,STORE,womens shorts,USD,RET,NaN,smkm,None,4,...,None,None,None,None,0.0,659.0,677.0,0.0,678.0,284.0


In [ ]:
# This query returns too much data for VSCODE it seems... This is the query to get raw data
raw_sql = ("""
WITH 

-- #CTE1: WOMENS TX BY CORE, NON CORE (CUSTOM DEFINITION), WITH PRODUCT INFO
  TX_FW AS (
    SELECT
      td.MASTER_GUEST_ID
      , td.TRANSACTION_ID
      , td.TXN_TIMESTAMP_PACIFIC
      , td.TXN_DATE_ID_PACIFIC
      , dd.FISCAL_WEEK
      , dd.FISCAL_YEAR_NUM
      , td.MASTER_ORDER_ORIGIN
      , td.SKU
      , td.CURRENCY_CODE
      , lower(pm.MMS_STYLE_DESCRIPTION) as MMS_STYLE_DESCRIPTION
      , lower(pm.MMS_DEPT_NAME3) as MMS_DEPT_NAME3
      , SPLIT_PART(lower(pm.FSO_COLORWAY_NAME), ' - ', 2) AS MMS_COLORWAY -- only get the part after the hyphen
      , lower(pm.FSO_COLORWAY_NAME) AS FSO_COLORWAY -- ORIGINAL
      , cs.SKU_COLOR_GROUP
      , cs.SKU_COLOR_CODE_DESC
      , cs.NUM_COLORS
      , cs.COLOR1
      , cs.COLOR2
      , cs.COLOR3
      , cs.COLOR4
      , cs.COLOR5
      , cs.SKU_SIZE AS FUS_SIZE
      , pm.MMS_SIZDSC as MMS_SIZE
      , CASE WHEN td.MASTER_GUEST_ID is null THEN 1 ELSE 0 END as no_id_guest
      , CASE WHEN contains(lower(pm.FSO_COLORWAY_NAME), 'blk') AND contains(lower(pm.MMS_STYLE_DESCRIPTION),'align') AND td.MMS_DEPT_NAME = 'LULU WOMENS' THEN 1 ELSE 0 END as MMS_BA_FLAG -- black align flag
      , CASE WHEN cs.SKU_COLOR_GROUP LIKE '%black%' AND contains(lower(pm.MMS_STYLE_DESCRIPTION),'align') AND td.MMS_DEPT_NAME = 'LULU WOMENS' THEN 1 ELSE 0 END as FUS_BA_FLAG -- black align flag 2
      , CASE WHEN contains(lower(pm.FSO_COLORWAY_NAME), 'blk') AND td.MMS_DEPT_NAME = 'LULU WOMENS' THEN 1 ELSE 0 END as MMS_B_FLAG -- black flag 1
      , CASE WHEN cs.SKU_COLOR_GROUP LIKE '%black%' AND pm.MMS_DEPT_NAME = 'LULU WOMENS' THEN 1 ELSE 0 END as FUS_B_FLAG -- black flag 2
      , CASE WHEN lower(pm.MMS_STYLE_DESCRIPTION) like '%align%pant%28%'
               or lower(pm.MMS_STYLE_DESCRIPTION) like '%align%pant%25%'
               or lower(pm.MMS_STYLE_DESCRIPTION) like '%wunder%train%25%'
               or lower(pm.MMS_STYLE_DESCRIPTION) like '%fast%free%25%'
             THEN 1 
             ELSE 0 
           END as WCORE_FLAG -- KEVINS DEFINITION
      , CASE WHEN td.TXN_SEQUENCE_ASC = 1 AND td.MMS_DEPT_NAME = 'LULU WOMENS' THEN 'ACQ' ELSE 'RET' END as TYPE
      , CASE WHEN td.MMS_DEPT_NAME = 'LULU WOMENS' THEN 1 ELSE 0 END as WOMENS

    FROM 
      MARTECH_DB.NA_ANALYTICS_SCH.FACT_TXN_DETAIL td
      LEFT JOIN MARTECH_DB.NA_ANALYTICS_SCH.DIM_PRODUCT_MASTER_MMS pm on td.SKU = pm.SKU
      LEFT JOIN GUEST_ANALYTICS_DB.EXPIRES_ON_20210131_SCH.FUSION_COLOURS_SIZES_QA cs on td.SKU = cs.SKU_ID_NUM
      LEFT JOIN MARTECH_DB.NA_LANDING_SCH.DIM_DATE dd on td.TXN_DATE_ID_PACIFIC = dd.DATE_ID

    WHERE 1 = 1
      AND dd.FISCAL_YEAR_NUM between 2015 and 2021
      AND dd.FISCAL_WEEK between 14 and 33
      AND td.TXN_SEQUENCE_ASC > 0 -- only look at txn_seq greater than 1, if removed will need to change TYPE logic
      AND td.MMS_DEPT_NAME = 'LULU WOMENS' -- WOMENS ONLY
    )


SELECT 
  t1.* -- just get all columns
  , max(t1.WCORE_FLAG) OVER (PARTITION BY t1.MASTER_GUEST_ID,t1.TYPE) as WCORE_COUNT
FROM 
  TX_FW t1;""")

## Part 2: Data-wrangling

In [15]:
# Are customers buying less colours? less sizes?

# Need to get data at size & colour granularity, following some bucketing of those features

# Data granularity: 
#   filters: womens tx only, fy 2019-2021, fw 14 - 44
#   time: Fiscal week... 
#   customer: type (of acquisition)... 
#   tx: channel, currency... 
#   product: dept, colour (mms & fusion), size (mms & fusion)... 
#   agg: no_id_guest (ct of null guests), guest_count (ct of guests), txn_count (distinct transactions), txn_wcore (womens core tx), txn_wrest (womens no core tx), guest_wcore (guests who bought womens core in this time period)

df.columns

Index(['FISCAL_YEAR_NUM', 'FISCAL_WEEK', 'MASTER_ORDER_ORIGIN',
       'MMS_DEPT_NAME3', 'CURRENCY_CODE', 'TYPE', 'NUM_COLORS', 'MMS_COLORWAY',
       'SKU_COLOR_GROUP', 'MMS_SIZE', 'FUS_SIZE', 'COLOR1', 'COLOR2', 'COLOR3',
       'COLOR4', 'COLOR5', 'NO_ID_GUEST', 'GUEST_COUNT', 'TXN_COUNT',
       'TXN_WCORE', 'TXN_WREST', 'GUEST_WCORE'],
      dtype='object')

In [23]:
# Here, we should explore the colour and size groupings...
# its not at the right level (should be product-sku), but thats ok... use this as directional
# once that is done, create the bins upstream and then come back and create the data viz

groupings =  ['SKU_COLOR_GROUP', 'FUS_SIZE']
aggregation = {TXN_COUNT: 'ncount'}
g = df.groupby('SKU_COLOR_GROUP', 'FUS_SIZE').agg('count').reset_index()

g

ValueError: No axis named FUS_SIZE for object type DataFrame